# A simple Text Classifier #
Author: Christin Seifert, licensed under the Creative Commons Attribution 3.0 Unported License https://creativecommons.org/licenses/by/3.0/ 
It is based on a tutorial of Nils Witt (https://github.com/n-witt/MachineLearningWithText_SS2017)


This is a tutorial for learning and evaluating a simple naive bayes classifier on for a simple text classification problem. In this tutorial you will:

* inspect the data you will be using to train the decision tree 
* train a decision tree 
* evaluate how well the decision tree does 
* visualize the decision tree

It is assumed that you have some general knowledge on 
* document-term matrices
* what a Naive Bayes classifier does

# Converting texts to features

We wil start with a small example of 3 SMS'. The texts in the SMS are the following "call me tonight", "Call me a cab", "please call me... PLEASE!" In order to do text classification we need to convert the text into a feature vector. We will follow a very simple approach here:
1. Find out which different words (or tokens) are used in the text. These makes up the vocabulary.
2. The length of a vector for each document then is the size of the vocabulary, and each entry in the vector corresponds to one word. This means, the first entry in the vector corresponds to the first word in the vocabulary, the second to the second and .. you get the logic ;-)
3. For each document we simply cound how often each word occurs and write it at the index in the vector that corresponds to this word. 

All those things can easily be done with the [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from the sklearn library.

In [9]:
simple_train = ['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

In [10]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
# vect = CountVectorizer()
vect = CountVectorizer(stop_words='english')

In [11]:
# learn the 'vocabulary' of the training data 
vect.fit(simple_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [12]:
# examine the fitted vocabulary
vect.get_feature_names()

['cab', 'tonight']

Have you noticed that all words are lower case now? And that we ignored punctuation? Whether this is a good idea, depends on the application. E.g. for detecting emotions in texts, smilies (punctutation) might be a helpful feature. But for now, let's keep it simple.

Now we generate a document-term matrix. In this matrix each row corresponds to one document, each column to one feature. Entry `(i,j)` tells us how often word `j` occurs in document `i`.

_Note:_ The "how often" is only true if we use the count vectorizer. Instead of word count there are many other possible features.

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> In this scheme, features and samples are defined as follows:

> - Each individual token occurrence frequency (normalized or not) is treated as a **feature**.
> - The vector of all the token frequencies for a given document is considered a **sample**.

> A **corpus of documents** can thus be represented by a matrix with **one row per document** and **one column per token** (e.g. word) occurring in the corpus.

> We call **vectorization** the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the **Bag of Words** or "Bag of n-grams" representation. Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [7]:
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

NotFittedError: Vocabulary not fitted or provided

In [51]:
# convert sparse matrix to a dense matrix
simple_train_dtm.toarray()

array([[0, 1],
       [1, 0],
       [0, 0]], dtype=int64)

We can use a pandas data frame to store the vector and the feature names together. 

In [9]:
# examine the vocabulary and document-term matrix together
import pandas as pd
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


Since in general this is an aweful lot of zeros (think of how many of all English words are present in a SMS), the more efficient way to store the information is as a sparse matrix. For humans this is a bit harder to read.

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> As most documents will typically use a very small subset of the words used in the corpus, the resulting matrix will have **many feature values that are zeros** (typically more than 99% of them).

> For instance, a collection of 10,000 short text documents (such as emails) will use a vocabulary with a size in the order of 100,000 unique words in total while each document will use 100 to 1000 unique words individually.

> In order to be able to **store such a matrix in memory** but also to **speed up operations**, implementations will typically use a **sparse representation** such as the implementations available in the `scipy.sparse` package.

In [10]:
# check the type of the document-term matrix
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [11]:
# examine the sparse matrix contents
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


### Generate the feature vector for a previously unseen text
In order to make predictions for unseen data, the new observation must have the same features as the training observations, both in number and meaning.

In [52]:
# example text for model testing
simple_test = ["please don't call me, I don't like you"]

In [53]:
# transform testing data into a document-term matrix (using existing vocabulary)
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 0]], dtype=int64)

In [54]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,cab,tonight
0,0,0


## A simple spam filter 
Now we are going to implement a simple spam filter for SMS messages. We are given a data set with SMS that are already annotated with either spam or ham (=not spam). We first load the data set and have a look at the data.

In [55]:
path = 'material/sms.tsv'
sms = pd.read_table(path, header=None, names=['label', 'message'])

In [56]:
sms.shape

(5572, 2)

In [57]:
# examine the first 10 rows
sms.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


We convert the label to a numerical value. 

In [18]:
# examine the class distribution
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [19]:
# convert label to a numerical variable
sms['label_num'] = sms.label.map({'ham':0, 'spam':1})

In [20]:
# check that the conversion worked
sms.head(10)

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


Now we have our text in the column `message` and our label in the column `label_num`. Let's have a look at the sizes. 

In [21]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = sms.message
y = sms.label_num
print(X.shape)
print(y.shape)

(5572,)
(5572,)


And at the text of the first 5 messages.

In [22]:
sms.message.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: message, dtype: object

We now prepare the data for the classifier. First split it into a training and a test set. There is a convenient method `train_test_split` available that helps us with that. We use a fixed random state `random_state=42`to split randomly, but at the same time get the same results each time we run the code.

In [6]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

NameError: name 'X' is not defined

Now we use the data preprocessing knowledge from above and generate the vocabulary. We will do this ONLY on the training data set, because we presume to have no knowledge whatsoever about the test data set. So we don't know the test data's vocabulary.

In [5]:
# learn training data vocabulary, then use it to create a document-term matrix
vect = CountVectorizer(stop_words='english',ngram_range=(1, 2), min_df=2,max_df=0.5)
print(X_train[:5])
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

NameError: name 'X_train' is not defined

In [79]:
# examine the document-term matrix
X_train_dtm

<4179x6920 sparse matrix of type '<class 'numpy.int64'>'
	with 39870 stored elements in Compressed Sparse Row format>

Next we transform the test data set using the same vocabulary (that is using the same `vect` object that internally knows the vocabulary).

In [80]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x6920 sparse matrix of type '<class 'numpy.int64'>'
	with 10621 stored elements in Compressed Sparse Row format>

### Building and evaluating a model

Now we are at the stage where we have a matrix of features and the corresponding labels. We can now train a classifier for spam detection on sms. We will use [multinomial Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html):

> The multinomial Naive Bayes classifier is suitable for classification with **discrete features** (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [81]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [82]:
nb.fit(X_train_dtm, y_train)
y_test_pred = nb.predict(X_test_dtm)

In [83]:
# from sklearn import metrics
metrics.accuracy_score(y_test, y_test_pred)

0.9877961234745154

In [84]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_test_pred)

array([[1201,    6],
       [  11,  175]], dtype=int64)

### "Spaminess" of words 

Before we start: the estimator has several fields that allow us to examine its internal state:

In [85]:
vect.vocabulary_

{'winner': 6681,
 'valued': 6375,
 'network': 4166,
 'customer': 1629,
 'selected': 5188,
 '900': 553,
 'prize': 4719,
 'reward': 5001,
 'claim': 1331,
 'code': 1377,
 'valid': 6368,
 '12': 182,
 'hours': 2931,
 'winner valued': 6684,
 'valued network': 6379,
 'network customer': 4167,
 'claim code': 1335,
 'valid 12': 6369,
 'hope': 2902,
 'showing': 5316,
 'care': 1195,
 'live': 3489,
 'dream': 1935,
 'derek': 1776,
 'class': 1345,
 'aight': 658,
 'lemme': 3400,
 'know': 3271,
 'lemme know': 3401,
 'yo': 6898,
 'watching': 6557,
 'movie': 4040,
 'watching movie': 6559,
 'don': 1880,
 'tell': 5809,
 'friend': 2359,
 'sure': 5718,
 'want': 6508,
 'smokes': 5435,
 'spend': 5546,
 'begging': 914,
 'come': 1417,
 'smoke': 5433,
 'don tell': 1894,
 'tell friend': 5813,
 'want live': 6517,
 'pain': 4422,
 'better': 937,
 'shall': 5272,
 'send': 5201,
 'mail': 3721,
 'id': 2982,
 'mail id': 3722,
 'ok': 4325,
 'just': 3177,
 'couple': 1560,
 'days': 1713,
 'lt': 3666,
 'couple days': 1561,
 

In [86]:
X_train_tokens = vect.get_feature_names()
print(X_train_tokens[:50])

['00', '00 sub', '000', '000 bonus', '000 cash', '000 homeowners', '000 pounds', '000 prize', '008704050406', '008704050406 sp', '02', '02 06', '0207', '0207 153', '02073162414', '02073162414 costs', '03', '03 05', '03 2nd', '04', '05', '050703', '050703 csbcm4235wc1n3xx', '06', '06 03', '07', '07 11', '07123456789', '07123456789 87077', '07734396839', '07734396839 ibh', '0776xxxxxxx', '0776xxxxxxx ve', '07781482378', '07781482378 com', '07xxxxxxxxx', '07xxxxxxxxx won', '08', '0800', '0800 1956669', '0800 542', '08000407165', '08000407165 18', '08000776320', '08000776320 reply', '08000839402', '08000839402 2stoptxt', '08000839402 call2optout', '08000930705', '08000930705 delivery']


In [87]:
print(X_train_tokens[-50:])

['year best', 'year old', 'year special', 'year supply', 'years', 'years old', 'years shower', 'yeh', 'yelling', 'yellow', 'yep', 'yer', 'yer mobile', 'yes', 'yes 440', 'yes 85023', 'yes callback', 'yes knw', 'yes princess', 'yes really', 'yes speak', 'yes tv', 'yest', 'yesterday', 'yesterday today', 'yetunde', 'yijue', 'ym', 'yo', 'yo yo', 'yoga', 'yogasana', 'yor', 'youre', 'yr', 'yr prize', 'yrs', 'yummy', 'yun', 'yun ah', 'yuo', 'yuo exmpel', 'yuo raed', 'yup', 'yup ok', 'yup thk', 'zed', 'zed 08701417012', 'zed 08701417012150p', 'zoe']


In [88]:
# feature count per class
nb.feature_count_

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 7.,  4., 22., ...,  3.,  2.,  1.]])

In [89]:
# number of times each token appears across all HAM messages
ham_token_count = nb.feature_count_[0, :]

# number of times each token appears across all SPAM messages
spam_token_count = nb.feature_count_[1, :]

In [90]:
# create a table of tokens with their separate ham and spam counts
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count, 'spam':spam_token_count}).set_index('token')
tokens.head()

,ham,spam
token,,
00,0.0,7.0
00 sub,0.0,4.0
000,0.0,22.0
000 bonus,0.0,6.0
000 cash,0.0,7.0


In [91]:
tokens.sample(5, random_state=6)

,ham,spam
token,,
summer free,0.0,2.0
1x150p,0.0,4.0
nw came,2.0,0.0
shock,1.0,1.0
200 free,0.0,2.0


Naive Bayes counts the number of observations in each class

In [92]:
nb.class_count_

array([3618.,  561.])

Add 1 to ham and spam counts to avoid dividing by 0

In [93]:
tokens['ham'] = tokens.ham + 1
tokens['spam'] = tokens.spam + 1
tokens.sample(5, random_state=6)

,ham,spam
token,,
summer free,1.0,3.0
1x150p,1.0,5.0
nw came,3.0,1.0
shock,2.0,2.0
200 free,1.0,3.0


In [94]:
# convert the ham and spam counts into frequencies
tokens['ham'] = tokens.ham / nb.class_count_[0]
tokens['spam'] = tokens.spam / nb.class_count_[1]
tokens.sample(5, random_state=6)

,ham,spam
token,,
summer free,0.000276,0.005348
1x150p,0.000276,0.008913
nw came,0.000829,0.001783
shock,0.000553,0.003565
200 free,0.000276,0.005348


Calculate the ratio of spam-to-ham for each token

In [95]:
tokens['spam_ratio'] = tokens.spam / tokens.ham
tokens.sample(5, random_state=6)

,ham,spam,spam_ratio
token,,,
summer free,0.000276,0.005348,19.347594
1x150p,0.000276,0.008913,32.245989
nw came,0.000829,0.001783,2.149733
shock,0.000553,0.003565,6.449198
200 free,0.000276,0.005348,19.347594


Examine the DataFrame sorted by spam_ratio

In [77]:
tokens.sort_values('spam_ratio', ascending=False)

,ham,spam,spam_ratio
token,,,
claim,0.000276,0.140820,509.486631
prize,0.000276,0.105169,380.502674
150p,0.000276,0.096257,348.256684
tone,0.000276,0.083779,303.112299
18,0.000276,0.071301,257.967914
...,...,...,...
later,0.030956,0.001783,0.057582
lor,0.031509,0.001783,0.056572
da,0.032338,0.001783,0.055121


In [43]:
tokens.loc['00', 'spam_ratio']

51.593582887700535

### Tuning the vectorizer
Do you see any potential to enhance the vectorizer? Think about the following questions:  
__Are all word equally important?__  
__Do you think there are "noise words" which negatively influence the results?__  
__How can we account for the order of words?__

#### Stopwords
Stopwords are the most common words in a language. Examples are 'is', 'which' and 'the'. Usually is beneficial to exclude these words in text processing tasks.  
The `CountVectorizer` has a `stop_words` parameter:
- **stop_words:** string {'english'}, list, or None (default)
    - If 'english', a built-in stop word list for English is used.
    - If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens.
    - If None, no stop words will be used.

In [44]:
vect = CountVectorizer(stop_words='english')

#### n-grams

n-grams concatenate n words to form a token. The following accounts for 1- and 2-grams

In [45]:
vect = CountVectorizer(ngram_range=(1, 2))

#### Document frequencies

Often it's beneficial to exclude words that appear in the majority or just a couple of documents. This is, very frequent or infrequent words. This can be achieved by using the `max_df` and `min_df` parameters of the vectorizer.

In [46]:
# ignore terms that appear in more than 50% of the documents
vect = CountVectorizer(max_df=0.5)

# only keep terms that appear in at least 2 documents
vect = CountVectorizer(min_df=2)

### A note on Stemming
* 'went' and 'go'  
* 'kids' and 'kid'  
* 'negative' and 'negatively'

__What is the pattern?__

The process of reducing a word to it's word stem, base or root form is called _stemming_. Scikit-Learn has no powerfull stemmer, but other libraries like the [NLTK](http://www.nltk.org/) have. 

# Tf-idf
* Tf-idf can be understood as a modification of the *raw term frequencies* (tf)
* The concept behind tf-idf is to downweight terms proportionally to the number of documents in which they occur.
* The idea is that terms that occur in many different documents are likely unimportant or don't contain any useful information for Natural Language Processing tasks such as document classification.

##  Explanation by example
Let consider a dataset containing 3 documents:

In [96]:
import numpy as np
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])

First, we will compute the _term frequency_ (alternatively: Bag-of-Words) $tf(t, d)$. $t$ is the number of times a term occures in a document $d$. Using Scikit-Learn we can quickly get those numbers:

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
tf = cv.fit_transform(docs).toarray()
tf

array([[0, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [1, 2, 1, 1, 1, 2, 1]], dtype=int64)

In [98]:
cv.vocabulary_

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}

Secondly, we introduce *inverse document frequency* ($idf$) by defining the term *document frequency* $\text{df}(d,t)$, which is simply the number of documents $d$ that contain the term $t$. We can then define the idf as follows:

$$\text{idf}(t) = log{\frac{n_d}{1+\text{df}(d,t)}},$$ 
where  
$n_d$: The total number of documents  
$\text{df}(d,t)$: The number of documents that contain term $t$.

Note that the constant 1 is added to the denominator to avoid a zero-division error if a term is not contained in any document in the test dataset.

Now, Let us calculate the idfs of the words "and", "is," and "shining:"

In [100]:
n_docs = len(docs)

df_and = 1
idf_and = np.log(n_docs / (1 + df_and))
print('idf "and": %s' % idf_and)

df_is = 3
idf_is = np.log(n_docs / (1 + df_is))
print('idf "is": %s' % idf_is)

df_shining = 2
idf_shining = np.log(n_docs / (1 + df_shining))
print('idf "shining": %s' % idf_shining)

idf "and": 0.4054651081081644
idf "is": -0.2876820724517809
idf "shining": 0.0


Using those idfs, we can eventually calculate the tf-idfs for the 3rd document:

$$\text{tf-idf}(t, d) = \text{tf}(t, d) \times \text{idf}(t),$$

In [101]:
print('Tf-idfs in document 3:\n')
print('tf-idf "and": %s' % (1 * idf_and))
print('tf-idf "is": %s' % (2 * idf_is))
print('tf-idf "shining": %s' % (1 * idf_shining))

Tf-idfs in document 3:

tf-idf "and": 0.4054651081081644
tf-idf "is": -0.5753641449035618
tf-idf "shining": 0.0


### Tf-idf in Scikit-Learn

In [102]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(smooth_idf=False, norm=None)
tfidf.fit_transform(tf).toarray()[-1][:3]

array([2.09861229, 2.        , 1.40546511])

__Wait! Those numbers aren't the same!__

Tf-idf in Scikit-Learn is calculated a little bit differently. Here, the `+1` count is added to the idf, whereas instead of the denominator if the df:

$$\text{idf}(t) = log{\frac{n_d}{\text{df}(d,t)}} + 1$$ 

In [103]:
tf_and = 1
df_and = 1 
tf_and * (np.log(n_docs / df_and) + 1)

2.09861228866811

In [104]:
tf_is = 2
df_is = 3 
tf_is * (np.log(n_docs / df_is) + 1)

2.0

In [105]:
tf_shining = 1
df_shining = 2 
tf_shining * (np.log(n_docs / df_shining) + 1)

1.4054651081081644

### Normalization

By default, Scikit-Learn performs a normalization. The most common way to normalize the raw term frequency is l2-normalization, i.e., dividing the raw term frequency vector $v$ by its length $||v||_2$ (L2- or Euclidean norm).

$$v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v{_1}^2 + v{_2}^2 + \dots + v{_n}^2}}$$

__Why is that useful?__

For example, we would normalize our 3rd document `'The sun is shining and the weather is sweet'` as follows:

In [106]:
tfidf = TfidfTransformer(use_idf=True, smooth_idf=False, norm='l2')
tfidf.fit_transform(tf).toarray()[-1][:3]

array([0.46572049, 0.44383662, 0.31189844])

### Smooth idf

We are not quite there. Sckit-Learn also applies smoothing, which changes the original formula as follows:

$$\text{idf}(t) = log{\frac{1 + n_d}{1+\text{df}(d,t)}} + 1$$ 

In [107]:
tfidf = TfidfTransformer(use_idf=True, smooth_idf=True, norm='l2')
tfidf.fit_transform(tf).toarray()[-1][:3]

array([0.40474829, 0.47810172, 0.30782151])